In [1]:
from pathlib import Path
import scanpy as sc
import re, sys
import pandas as pd

In [2]:
def parse_sam(file):
	result = set()
	with open(file, 'r') as inputFile:
		for line in inputFile.readlines():
			fields = line.split('\t')
			acc = fields[2]
			for item in fields[19:]:
				if item.find('CB:Z:') > -1:
					barcode = re.sub(r'CB:Z:', r'', item)
				if item.find('UB:Z:') > -1:
					umi = re.sub(r'UB:Z:', '', item)
			result.add(f'{acc}\t{barcode}\t{umi}')
	return result

def generate_table(data, target_set, target_name, suffix):
	filtered_data = []
	for item in data:
		if item.split('\t')[0] in target_set:
			filtered_data.append(item)

	with open('exp_aae_denv.txt', 'w') as outputFile:
		for item in filtered_data:
			outputFile.write(item)
	table = pd.read_table('exp_aae_denv.txt', header=None, delimiter='\t')
	table.columns = ['Accession', 'Barcode', 'UMI']
	table['Accession'] = table['Accession'].map(lambda x: target_name[x])
	table = table[table['Barcode'] != '-']
	table = table[table['UMI'] != '-']
	table = table.groupby(['Accession', 'Barcode'])['UMI'].count().reset_index()
	table['Barcode'] = table['Barcode'] + suffix
	Path('exp_aae_denv.txt').unlink()
	return table

def write2csv(adata, table, target_name, suffix):
	umap_df = pd.DataFrame(
	adata.obsm['X_umap'], 
	index=adata.obs_names,
	columns=['UMAP_1', 'UMAP_2']  # 列名可自定义
	)
	umap_df = pd.concat([adata.obs['cluster'], umap_df], axis=1).reset_index()
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2']
	umap_df.to_csv('temp.csv', header=True, index=False)
	umap_df = pd.read_csv('temp.csv', header=0)
	for name in target_name.values():
		temp_column = table[table['Accession'] == name][['Barcode', 'UMI']]
		umap_df = umap_df.merge(temp_column, how='left', on='Barcode').fillna(0)
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2'] + list(target_name.values())
	umap_df.to_csv(f'exp_{suffix}.csv', index=False)
	Path('temp.csv').unlink()
	print(umap_df)

In [4]:
data = parse_sam('exp_cxpip.sam')

In [8]:
target_set = ['MW174761.1', 'ON949933.1']
target_name = {
	'MW174761.1': 'Totichi',
	'ON949933.1': 'HKIFV'
}
table = generate_table(data, target_set, target_name, '')

In [9]:
adata = sc.read('../../../UMAP_h5ad/cxpip.h5ad')
adata.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,mt_outlier,genes_outlier,leiden_res,cluster,cluster_name
AACACACAGAACGCTAGTTGCTTATGG,408,6.013715,1350.0,7.208601,29.407407,56.0,4.043051,4.148148,False,False,1,ISC/EB,ISC/EB
AACACACAGACACCAACGGAACCTGTA,853,6.749931,4347.0,8.377471,27.950311,111.0,4.718499,2.553485,False,False,16,ISC/EB-prol,ISC/EB-prol
AACACACAGACCAGGTCAAAGGAGTCT,377,5.934894,1004.0,6.912743,26.195219,29.0,3.401197,2.888446,False,False,10,Cardia-2,Cardia-2
AACACACAGACCAGGTCACGAAGGCAT,1166,7.062192,4637.0,8.442039,28.919560,415.0,6.030685,8.949752,False,False,13,EC-like-3,EC-like-3
AACACACAGACCTCGACTCAAGAGTAG,299,5.703782,661.0,6.495265,26.172466,10.0,2.397895,1.512859,False,False,6,EC-like-2,EC-like-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTGGACACTTGCCGTCACTCTAACAC,720,6.580639,2341.0,7.758760,25.715506,38.0,3.663562,1.623238,False,False,11,ISC/EB-prol,ISC/EB-prol
TGTGGACACTTGCCGTCAGAGTGATCT,347,5.852202,1027.0,6.935370,33.787731,20.0,3.044523,1.947420,False,False,17,VM,VM
TGTGGACACTTGCCGTCAGAGTTCGTC,1147,7.045777,11729.0,9.369905,42.492966,395.0,5.981414,3.367721,False,False,4,EC,EC
TGTGGACACTTGGTGACCCCGTGTCAA,808,6.695799,3919.0,8.273847,27.915285,127.0,4.852030,3.240623,False,False,2,ISC/EB,ISC/EB


In [10]:
adata = sc.read('../../../UMAP_h5ad/cxpip.h5ad')
# adata = adata[adata.obs['batch'] == 'cxtri']
write2csv(adata, table, target_name, 'cxpip')

                          Barcode      Cluster     UMAP_1    UMAP_2  Totichi  \
0     AACACACAGAACGCTAGTTGCTTATGG       ISC/EB   9.102228  5.040160      0.0   
1     AACACACAGACACCAACGGAACCTGTA  ISC/EB-prol  11.720303  9.595119      0.0   
2     AACACACAGACCAGGTCAAAGGAGTCT     Cardia-2   6.932008  3.808001      0.0   
3     AACACACAGACCAGGTCACGAAGGCAT    EC-like-3   1.543293  6.149855      0.0   
4     AACACACAGACCTCGACTCAAGAGTAG    EC-like-2   5.051599  2.745352      0.0   
...                           ...          ...        ...       ...      ...   
9839  TGTGGACACTTGCCGTCACTCTAACAC  ISC/EB-prol  12.071262  8.178706      0.0   
9840  TGTGGACACTTGCCGTCAGAGTGATCT           VM   4.354169  1.227471      0.0   
9841  TGTGGACACTTGCCGTCAGAGTTCGTC           EC  -1.755991  4.855513      0.0   
9842  TGTGGACACTTGGTGACCCCGTGTCAA       ISC/EB  11.970945  6.706281      0.0   
9843  TGTGGACACTTGGTGACCGGAGAGGAA    EC-like-2   6.309640  2.685626      0.0   

      HKIFV  
0       0.0  
1       0.0